###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Policía Nacional de Colombia

In [ ]:
import pandas as pd

###Archivo 2iz5-9bbz.csv

Contiene  información de Incautación de Armas de Fuego desde 01 de enero del año 2010 al 31 de enero del año 2024

In [ ]:
df = pd.read_csv("https://www.datos.gov.co/resource/2iz5-9bbz.csv?$limit=300000")
df.head()

,departamento,municipio_hecho,codigo_dane,clase_bien,fecha_hecho,cantidad
0,ANTIOQUIA,FREDONIA,5282000,ESCOPETA,01/01/2010,1
1,ATLÁNTICO,BARRANQUILLA (CT),8001000,ESCOPETA,01/01/2010,1
2,BOLÍVAR,MARÍA LA BAJA,13442000,ESCOPETA,01/01/2010,1
3,BOYACÁ,SUTATENZA,15778000,ESCOPETA,01/01/2010,1
4,CESAR,AGUACHICA,20011000,ESCOPETA,01/01/2010,1


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281479 entries, 0 to 281478
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   departamento     281479 non-null  object
 1   municipio_hecho  281479 non-null  object
 2   codigo_dane      281479 non-null  int64 
 3   clase_bien       281479 non-null  object
 4   fecha_hecho      281479 non-null  object
 5   cantidad         281479 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 12.9+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [ ]:
relevant_cols = ['codigo_dane', 'departamento', 'municipio_hecho', 'fecha_hecho', 'clase_bien', 'cantidad']
weapons_confiscation = df[relevant_cols]
weapons_confiscation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281479 entries, 0 to 281478
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   codigo_dane      281479 non-null  int64 
 1   departamento     281479 non-null  object
 2   municipio_hecho  281479 non-null  object
 3   fecha_hecho      281479 non-null  object
 4   clase_bien       281479 non-null  object
 5   cantidad         281479 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 12.9+ MB


### Convertir columna fecha_hecho a tipo date

In [ ]:
# Hacer una copia explícita del DataFrame
weapons_confiscation_ = weapons_confiscation.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime usando el formato correcto
weapons_confiscation_['fecha_hecho'] = pd.to_datetime(weapons_confiscation_['fecha_hecho'], format='%d/%m/%Y', errors='coerce', dayfirst=True)

# Extraer solo el año
weapons_confiscation_.loc[:, 'fecha_hecho'] = weapons_confiscation_['fecha_hecho'].dt.year

### Verificar valores nulos

In [ ]:
weapons_confiscation_.isnull().sum()

,0
codigo_dane,0
departamento,0
municipio_hecho,0
fecha_hecho,0
clase_bien,0
cantidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['clase_bien', 'departamento', 'municipio_hecho']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(weapons_confiscation_[column].unique())
    print()


Categorías en la columna 'clase_bien':
['ESCOPETA' 'PISTOLA' 'REVOLVER' 'SUBAMETRALLADORA' 'CARABINA'
 'FUSIL / RIFLE' 'AMETRALLADORA' 'LANZA GRANADAS' 'MORTERO' 'ROCKET'
 'LANZA MISIL' 'LANZA COHETES' 'ESCOPETA DE FISTO' 'ESCOPETA TRAUMATICA'
 'PISTOLA TRAUMATICA' 'REVOLVER TRAUMATICO' 'SUBAMETRALLADORA TRAUMATICA']

Categorías en la columna 'departamento':
['ANTIOQUIA' 'ATLÁNTICO' 'BOLÍVAR' 'BOYACÁ' 'CESAR' 'CÓRDOBA'
 'CUNDINAMARCA' 'HUILA' 'MAGDALENA' 'NARIÑO' 'SANTANDER' 'SUCRE' 'VALLE'
 'ARAUCA' 'CASANARE' 'CAUCA' 'META' 'NORTE DE SANTANDER' 'PUTUMAYO'
 'RISARALDA' 'TOLIMA' 'CALDAS' 'GUAJIRA' 'GUAVIARE' 'QUINDÍO' 'CAQUETÁ'
 'SAN ANDRÉS' 'CHOCÓ' 'VICHADA' 'VAUPÉS' 'AMAZONAS' 'GUAINÍA']

Categorías en la columna 'municipio_hecho':
['FREDONIA' 'BARRANQUILLA (CT)' 'MARÍA LA BAJA' ... 'Cabrera' 'Cachirá'
 'La Belleza']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    weapons_confiscation_[col] = weapons_confiscation_[col].apply(remove_accents_and_special_chars)

### Ajustar columna 'codigo_dane' para que coincida con el campo 'dept_mpio_code' de la tabla municipalities de la base de datos, que guarda toda la informacion de georeferenciacion de los municipios

 -  Cargar los datos con códigos reales de los municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE (Estos codigos son los reales)

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigo_dane" en el df weapons_confiscation_

In [ ]:
# Asegurarnos de que todos los valores en 'codigo_dane' sean strings
weapons_confiscation_['codigo_dane'] = weapons_confiscation_['codigo_dane'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = weapons_confiscation_['codigo_dane'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [ ]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [7 8]


In [ ]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,no_digitos_codigo_dane,count,percentage
0,8,228742,0.812643
1,7,52737,0.187357


In [ ]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = weapons_confiscation_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigo_dane']])
    print()

Muestra de registros con longitud 8:
  codigo_dane
2    13442000
3    15778000
4    20011000
5    23189000
6    11001000

Muestra de registros con longitud 7:
   codigo_dane
0      5282000
1      8001000
19     5030000
20     5360000
21     5001000



### Nota:

De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 81,26 tiene 8 dígitos (al parecer se le adicionaron 3 ceeros al final del código que realmente es de 5 dígitos), el 18,74% tiene solamente 7 dígitos (al parecer el cero a la izquierda de los códigos se suprimió).

  - Verificar si los codigo_dane de 7 dígitos corresponden a departamentos que se identifican con 1 dígito para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [ ]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtrado = weapons_confiscation_[weapons_confiscation_['codigo_dane'].str.len() == 7]

# Obtener las categorías únicas de la columna 'departamento'
categorias_departamento = filtrado['departamento'].unique()

# Imprimir las categorías
print(categorias_departamento)

['ANTIOQUIA' 'ATLANTICO']


Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane de un dígito, 5 y 8 respectivamente.

  - Adicionar un cero a los codigo_dane de 7 dígitos

In [ ]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_7(codigo):
    if len(codigo) == 7:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
weapons_confiscation_['codigo_dane'] = weapons_confiscation_['codigo_dane'].apply(add_zero_if_length_7)

In [ ]:
# Verificar cuantos codigo_dane de 7 dígitos quedaron
len(weapons_confiscation_[weapons_confiscation_['codigo_dane'].str.len() == 7])

0

  - Quitar los ultimos 3 ceros de los codigo_dane que tienen 8 dígitos

In [ ]:
# Función que corta los últimos 3 caracteres si la longitud del string es 8
def trim_last_3_if_length_8(codigo):
    if len(codigo) == 8:
        return codigo[:5]  # Dejar solo los primeros 5 caracteres
    return codigo

# Aplicar la función a la columna 'codigo'
weapons_confiscation_['codigo_dane'] = weapons_confiscation_['codigo_dane'].apply(trim_last_3_if_length_8)

In [ ]:
# Verificar cuantos codigo_dane de 8 dígitos quedaron
len(weapons_confiscation_[weapons_confiscation_['codigo_dane'].str.len() == 8])

0

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(weapons_confiscation_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en weapons_confiscation_", "Departamentos dept_mpios_codes")

Departamentos en weapons_confiscation_ que no están en Departamentos dept_mpios_codes:
{'GUAJIRA', 'VALLE', 'SAN ANDRES'}


  - Solucionar discrepancias en nombres de departamento

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                 'VALLE': 'VALLE DEL CAUCA',
                 'GUAJIRA':'LA GUAJIRA'}

# Reemplazar los nombres incorrectos  el dataframe weapons_confiscation_
weapons_confiscation_['departamento'] = weapons_confiscation_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas de departamento
compare_lists(weapons_confiscation_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en weapons_confiscation_", "Departamentos dept_mpios_codes")

Departamentos en weapons_confiscation_ que no están en Departamentos dept_mpios_codes:
set()


  - Verificar que todos los codigos esten estandarizados a 5 dígitos

In [ ]:
weapons_confiscation_['codigo_dane'].apply(len).nunique()

1

In [ ]:
weapons_confiscation_['codigo_dane'].apply(len).value_counts()

,count
codigo_dane,
5,281479


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [ ]:
# Comparar listas de códigos
compare_lists(weapons_confiscation_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios  weapons_confiscation_", "Códigos de municipios  dept_mpios_codes")

Departamentos en weapons_confiscation_ que no están en Departamentos dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = [ 'departamento', 'municipio_hecho']
final_weapons_confiscation = weapons_confiscation_.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_weapons_confiscation['source_id'] = 9

In [ ]:
final_weapons_confiscation.columns

Index(['codigo_dane', 'fecha_hecho', 'clase_bien', 'cantidad', 'source_id'], dtype='object')

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'fecha_hecho': 'year_of_incident',
    'clase_bien': 'type_of_weapon',
    'cantidad': 'amount',
    'codigo_dane': 'dane_code',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_weapons_confiscation.rename(columns=translation_map, inplace=True)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
final_weapons_confiscation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281479 entries, 0 to 281478
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   dane_code         281479 non-null  object
 1   year_of_incident  281479 non-null  int32 
 2   type_of_weapon    281479 non-null  object
 3   amount            281479 non-null  int64 
 4   source_id         281479 non-null  int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 9.7+ MB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_weapons_confiscation.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/weapons_confiscation.csv', index=False)